In [9]:
import openai
import random
import jsonlines

# Initialize OpenAI API key
openai.api_key = ""

# Load input text file
with open("input_text.txt", "r") as file:
    input_text = file.read()

# Define function to generate responses using ChatGPT
def generate_response(prompt):
    # Split prompt into multiple chunks with a maximum length of 4096 tokens
    chunks = [prompt[i:i + 4096] for i in range(0, len(prompt), 4096)]
    response = ""

    # Generate response for each chunk using ChatGPT
    for chunk in chunks:
        response += openai.Completion.create(
            engine="text-davinci-002",
            prompt=chunk,
            max_tokens=128,
            n=1,
            stop=None,
            temperature=0.5
        ).choices[0].text.strip()

    return response

# Generate settings, characters, types, and dialogue using ChatGPT
settings = [generate_response("Generate a setting for a comic strip:\n" + input_text) for i in range(5)]
characters = [generate_response("Generate a character for a comic strip:\n" + input_text) for i in range(5)]
types = [generate_response("Generate a character type for a comic strip:\n" + input_text) for i in range(5)]
dialogue = [generate_response("Generate some dialogue for a comic strip:\n" + input_text) for i in range(5)]

# Generate prompts and responses and write to JSONL file
with jsonlines.open('comic_strip_prompts_gpt.jsonl', mode='w') as writer:
    for i in range(5):
        # Generate prompt
        if i % 3 == 0:
            prompt = "Panel " + str(i // 3 + 1) + ": Describe the setting for the panel."
            response = random.choice(settings)
        elif i % 3 == 1:
            prompt = "Panel " + str(i // 3 + 1) + ": Create a character for the panel."
            response = random.choice(characters) + ", a " + random.choice(types)
        else:
            prompt = "Panel " + str(i // 3 + 1) + ": Write some dialogue for the panel."
            response = random.choice(characters) + ": " + random.choice(dialogue)

        # Write prompt and response to JSONL file
        writer.write({
            'prompt': prompt,
            'completion': response
        })
